import packages

In [1]:
import SciNN_BCM as sBCM
import SciNN_backprop as sbp
import numpy as np
import util
import os
# from IPython.display import clear_output
import copy
import random
import csv
import matplotlib.pyplot as plt
import pngToMp4 as p

set factors

In [2]:
TRAINING_SENARIO_NO = 2 # 0 ~ 5
# 0 : backprop을 통한 gamma -> 10us f 학습
# 1 : 대조군 평가 : gamma ~> r
# 2 : 붉은색 공포기억 연합화 : r + gamma ~> r
# 3 : 붉은색 공포기억 지우기 : r + gamma ~> gamma ~> r
# 4 : 붉은색 공포기억 덮어씌우기 : r + gamma ~> b + gamma ~> r or b
# 5 : 붉은색 앞으로 가기 학습 : r or r + gamma ~> r

DATA_FILE_TO_USE = "20240115_test_heatmap"
INPUT_COLOR_DIMENSION = 3  # 변경하면 안됨 (r, g, b)
INPUT_SHOCK_DIMENSION = 1 # (gamma)
INPUT_ENV_DIMENSION = 1
OUTPUT_MOVEMENT_DIMENSION = 4  # 변경하면 안됨 (f, r, up, down)
OUPUT_IRRELAVENT_DIMENSION = 1

INTERNAL_NEURON_LENGTH = 20
TRAIN_EPOCH = 400
TEST_EPOCH = 100
EXPERIMENT_EPOCH = 50


INPUT_ENV_VOLATILITY = 0.01

INITIAL_SPARSITY = 2.27

LIGHT_EXPOSURE_TIME = 20
WAITING_TIME = 10
FREEZE_TIME = 10

SHOCK_EXPOSURE_PROB = 1/2
RED_COLOR_EXPOSURE_PROB = 1/3
BLUE_COLOR_EXPOSURE_PROB = 1/3 # < 1 - RED_COLOR_EXPOSURE_PROB

CORRECT_LOSS_LIMIT = 0.05

SAVE_HEATMAP = True # Heatmap 저장 및 영상 생성 여부, False인 경우 속도 향상됨

SAVE_LEARNING_HISTORY_FILE_NAME = "backprop fear creation 3" + f"[{TRAINING_SENARIO_NO}]"
SAVE_HEATMAP_FILE_NAME = "backprop fear creation 3" + \
    f"[{TRAINING_SENARIO_NO}]"

if not os.path.isdir(f"output/{SAVE_HEATMAP_FILE_NAME}"):
    os.mkdir(f"output/{SAVE_HEATMAP_FILE_NAME}")

set new PANN model

In [3]:
output_length = OUTPUT_MOVEMENT_DIMENSION + OUPUT_IRRELAVENT_DIMENSION
input_ext_length = INPUT_COLOR_DIMENSION + INPUT_SHOCK_DIMENSION + INPUT_ENV_DIMENSION
input_length = input_ext_length + output_length

a = sBCM.SciNN_BCM(input_length)
if TRAINING_SENARIO_NO == 0:
    a = sbp.SciNN_backprop(input_length)
a.addLayer(INTERNAL_NEURON_LENGTH)
a.addLayer(output_length)
if TRAINING_SENARIO_NO == 0:
    a.setActivationFunction("tanh")
    a.setLossFunction("mse")
a.initializeWeight(INITIAL_SPARSITY)
print(a.W)

[[[0.04383861916120746, 0, 0, 0, 0.36808818888647976, 0, 0.2747723912294179, 0, 0, 0, 0, -0.22985205535560338, 0, 0, 0, 0.26375684948517064, 0, 0, 0.4656251806103652, 0.44312667010878914, -0.20053954309543987, 0, 0.37977769298044073, 0, 0, 0, 0, 0, 0.1259546375175119, -0.028790306467088023], [0, 0, 0, 0, 0, 0, 0, 0, 0, -0.15207083719688652, 0, 0, 0, 0, 0.4440594331973612, -0.36465122889366586, 0.32801544113931924, -0.4845889959055558, 0, 0, 0.4586724804248292, -0.3623515559678817, 0, 0, 0, 0.008089037641764607, 0.01642455939940124, 0, 0, -0.4000355419998619], [0, 0, 0, 0, -0.05598372128218221, -0.05184882323599849, 0, 0, 0, 0, -0.24674747219488646, -0.23198476174762483, 0, -0.11991973839372061, -0.267986832678874, 0, 0.1886827299740561, -0.0983074304961592, 0, 0.22196084520810844, -0.2893882219140613, 0.28958650616637116, 0.0474374876845437, 0, 0, 0.10682386122035348, -0.38828886728412815, 0.42760107928054836, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0.3512145150188509, -0.35653805018936136, 0.

set random input-target generating function

In [4]:
temp_red_prob = RED_COLOR_EXPOSURE_PROB - (1-RED_COLOR_EXPOSURE_PROB-BLUE_COLOR_EXPOSURE_PROB)/25
temp_blue_prob = BLUE_COLOR_EXPOSURE_PROB - (1-RED_COLOR_EXPOSURE_PROB-BLUE_COLOR_EXPOSURE_PROB)/25

def generateColor():
    p = random.random()
    if p < temp_red_prob:
        return {"input_color": [1-random.random()/3, random.random()/3, random.random()/3], "label": "red"}
    elif p < temp_red_prob + temp_blue_prob:
        return {"input_color": [random.random()/3, random.random()/3, 1-random.random()/3], "label": "blue"}
    else:
        input_color = [random.random(), random.random(), random.random()]
        label = ""
        if input_color[0] > 2/3 and input_color[1] < 1/3 and input_color[2] < 1/3:
            label = "red"
        elif input_color[0] < 1/3 and input_color[1] < 1/3 and input_color[2] > 2/3:
            label = "blue"
        else:
            label = "others"
        return {"input_color": input_color, "label": label}

create fear reaction

In [5]:
if TRAINING_SENARIO_NO == 0:

    output_vector_list_prev = [
        [0 for i in range(output_length)] for l in range(a.layer_depth+1)]
    f = open(f"output/{SAVE_LEARNING_HISTORY_FILE_NAME}.csv",
             "w", encoding="utf-8", newline="")
    wr = csv.writer(f)
    wr.writerow(["no", "iteration", "input_shock", "time", "loss", "success", "correct_count"])
    
    for ep in range(TRAIN_EPOCH + TEST_EPOCH):
        shock_now = random.random() < SHOCK_EXPOSURE_PROB
        correct_count = 0
        shock_stopped = False
        input_color = [random.random() for color in range(INPUT_COLOR_DIMENSION)]
        input_env = [random.random() for env in range(INPUT_ENV_DIMENSION)]
        
        for t in range(LIGHT_EXPOSURE_TIME + WAITING_TIME):
            output_vector_prev = output_vector_list_prev.pop(0)
            input_env = [input_env[env] + 2*random.random()*INPUT_ENV_VOLATILITY -
                         INPUT_ENV_VOLATILITY for env in range(INPUT_ENV_DIMENSION)]
            input_color = [input_color[color] + 2*random.random()*INPUT_ENV_VOLATILITY -
                         INPUT_ENV_VOLATILITY for color in range(INPUT_COLOR_DIMENSION)]
            if t < LIGHT_EXPOSURE_TIME:
                input_vector = [
                    *input_color, *[1 if shock_now else 0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env, *output_vector_prev]
            else : 
                input_vector = [
                    *input_color, *[0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env, *output_vector_prev]
                
            target_vector = [1 if shock_now else 0, 0 if shock_now else 1] 
            
            a.propagateForward(input_vector)
            
            if a.lossFn(a.h[-1][0:2], target_vector) < CORRECT_LOSS_LIMIT:
                if correct_count > FREEZE_TIME:
                    shock_stopped = True
                else:
                    correct_count += 1
            
            if not shock_stopped and ep < TRAIN_EPOCH:
                a.propagateBackward(
                    [*target_vector, *a.h[-1][2:]])
            util.printAfterErase(
                f"iteration : {ep+1}; time : {t+1} ; loss : {a.lossFn(a.h[-1][0:2],target_vector)} ({a.h[-1][0:2]}<->{target_vector})      ")
            wr.writerow([ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,
                        ep+1,
                        shock_now,
                        t+1,
                        a.lossFn(a.h[-1][0:2],target_vector),
                        1 if shock_stopped else 0,
                        correct_count
                        ])
            if SAVE_HEATMAP:
                a.saveHeatmap(f"output/{SAVE_HEATMAP_FILE_NAME}/{util.fillZero(ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,5)}.png")
            output_vector_list_prev.append(copy.deepcopy(a.h[-1]))

    f.close()
    a.saveCurrent(DATA_FILE_TO_USE)
    if SAVE_HEATMAP:
        p.pngToMp4(f"output/{SAVE_HEATMAP_FILE_NAME}",f"output/{SAVE_HEATMAP_FILE_NAME}")
    print("\nDone")
    # clear_output()

iteration : 500; time : 30 ; loss : 0.916631664263051 ([0, 0.9128325851579259]<->[1, 0])       4]<->[1, 0])           
Done


In [6]:
a.loadData(DATA_FILE_TO_USE)
print(a.W)

[[[0.08593710751967312, 0.04399206276061214, 0.03844146531485905, 0.1304602577856477, 0.40074057449176614, 0.04526697436276489, 0.31430303513652885, 6.981601550854437e-07, 0.02885946018703302, 0.011485571958013445, 0, -0.1960643110443757, 0.00046773824466028697, 0.051836368437979424, 0.06403118781562042, 0.2660792370888234, 0.049262583266849944, 0.00011740683680225994, 0.465779689526467, 0.45667668860618255, -0.19032658910962633, 9.601767521545252e-05, 0.41850124510946246, 0.006560471182292453, 0.029393492268542494, 0.020873448984403575, 0.03307940153648309, 0.04639269829925236, 0.16824220095211703, 0.01419024703036144], [0.02387530408256952, 0.02862217521388346, 0.028191583419970734, 0.02737266909035093, 0.021419525886466975, 0.014787094408903993, 0.03608227285422779, 7.216042330903655e-06, 0.010715525636373466, -0.14159791766497287, 0.02889009471748726, 0, 0.00024168421267022218, 0.02648819544098287, 0.472684163653872, -0.36344977148140745, 0.3499148549198303, -0.4845805709530281, 3.

connect fear with red light

In [7]:
# color : "red" | "blue" | "random"
def _mseLoss(output, target):
    return sum([(output[i]-target[i])**2 for i in range(len(output))])/len(output)

def trainColorAndShockExperiment(file_index=0, color="red", use_shock=True, reset=False):
    if reset:
        a.loadData(DATA_FILE_TO_USE)
        a.decodeWeightOfBackprop()
        a.saveCurrent("temp")
    if not os.path.isdir(f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]") and SAVE_HEATMAP:
        os.mkdir(f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]")
    
    input_env = [random.random() for env in range(INPUT_ENV_DIMENSION)]
    output_vector_list_prev = [
        [0 for i in range(output_length)] for l in range(a.layer_depth+1)]
    f = open(f"output/{SAVE_LEARNING_HISTORY_FILE_NAME}[{file_index}].csv",
             "w", encoding="utf-8", newline="")
    wr = csv.writer(f)
    wr.writerow(["no", "iteration", "color", "time",
                "loss", "success", "correct_count", "shock", "freeze", "release"])

    for ep in range(EXPERIMENT_EPOCH):
        input_color_sample = generateColor()
        rand_val = random.random()
        is_correct_color = (input_color_sample["label"] == color) or (color == "random" and (rand_val < 0.5)) 
        shock_now = use_shock and is_correct_color
        correct_count = 0
        target_vector = [1 if is_correct_color else 0, 0 if is_correct_color else 1] 

        for t in range(LIGHT_EXPOSURE_TIME + WAITING_TIME):
            output_vector_prev = output_vector_list_prev.pop(0)
            input_env = [input_env[env] + 2*random.random()*INPUT_ENV_VOLATILITY -
                         INPUT_ENV_VOLATILITY for env in range(INPUT_ENV_DIMENSION)]
            if t < LIGHT_EXPOSURE_TIME:
                input_vector = [
                    *input_color_sample["input_color"], *[1 if shock_now else 0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env, *output_vector_prev]
            else:
                input_vector = [
                    *[0 for c in range(INPUT_COLOR_DIMENSION)], *[0 for s in range(INPUT_SHOCK_DIMENSION)], *input_env, *output_vector_prev]

            a.propagate(input_vector)

            if _mseLoss(a.h[-1][0:2], target_vector) < CORRECT_LOSS_LIMIT:
                if correct_count < FREEZE_TIME:
                    correct_count += 1

            util.printAfterErase(
                f"iteration : {ep+1}; time : {t+1} ; loss : {_mseLoss(a.h[-1][0:2], target_vector)}   {rand_val}  ")
            wr.writerow([ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,
                        ep+1,
                        input_color_sample['label'],
                        t+1,
                        _mseLoss(a.h[-1][0:2], target_vector),
                        correct_count>=FREEZE_TIME,
                        correct_count,
                        shock_now,
                        *a.h[-1][0:2]
                         ])
            if SAVE_HEATMAP:
                a.saveHeatmap(
                    f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]/{util.fillZero(ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,5)}.png")
            output_vector_list_prev.append(copy.deepcopy(a.h[-1]))

    f.close()
    
    if SAVE_HEATMAP:
        p.pngToMp4(f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]",f"output/{SAVE_HEATMAP_FILE_NAME}[{file_index}]")
    
    print("\nDone")
    # clear_output()

In [8]:

if TRAINING_SENARIO_NO == 1:
    trainColorAndShockExperiment(0, "random", True, True)
    trainColorAndShockExperiment(1, "red", False)
if TRAINING_SENARIO_NO == 2:
    trainColorAndShockExperiment(0,"red",True, True)
    a.saveCurrent("temp2")
    trainColorAndShockExperiment(1, "red",False)
    a.saveCurrent("temp3")

activate model

In [9]:
if False:

    input_color = []
    input_env = [random.random() for env in range(INPUT_ENV_DIMENSION)]
    # t-L-1 ~ t-1 범위의 과거 output 저장
    output_vector_list_prev = [[0 for i in range(output_length)] for l in range(a.layer_depth+1)]
    f = open(f"output/{SAVE_LEARNING_HISTORY_FILE_NAME}.csv","w",encoding="utf-8",newline="")
    wr = csv.writer(f)
    wr.writerow(["no", "iteration", "input_color", "input_env", "time", "loss", "success", "correct_count"])

    for ep in range(TRAIN_EPOCH + TEST_EPOCH):
        input_target_sample = generateFn()
        input_color = [random.random() for color in range(INPUT_COLOR_DIMENSION)]
        correct_count = 0
        feed_stopped = False
        
        for t in range(LIGHT_EXPOSURE_TIME + WAITING_TIME):
            output_vector_prev = output_vector_list_prev.pop(0)
            input_env = [input_env[env] + 2*random.random()*INPUT_ENV_VOLATILITY - INPUT_ENV_VOLATILITY for env in range(INPUT_ENV_DIMENSION)]
            
            if t < LIGHT_EXPOSURE_TIME:
                input_vector = [*input_target_sample["input_color"], *input_env, *output_vector_prev]
            else:
                input_vector = [*[0 for color in range(INPUT_COLOR_DIMENSION)], *input_env, *output_vector_prev]
            
            target_vector = input_target_sample["target_vector"]
            
            a.propagateForward(input_vector)
            
            if a.lossFn(a.h[-1],[*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]]) < CORRECT_LOSS_LIMIT:
                if correct_count > LIGHT_EXPOSURE_TIME:
                    feed_stopped = True
                else:
                    correct_count += 1
                        
            if not feed_stopped and ep < TRAIN_EPOCH:
                a.propagateBackward([*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]])
                
            util.printAfterErase(f"iteration : {ep+1}; time : {t+1} ; loss : {a.lossFn(a.h[-1],[*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]])}     ")
            wr.writerow([ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,
                        ep+1,
                        target_vector[0]+2*target_vector[1], # none: 0, r: 1, b: 2
                        input_env[0],
                        t+1,
                        a.lossFn(a.h[-1],[*target_vector, *a.h[-1][OUPUT_MOVEMENT_DIMENSION:]]),
                        1 if feed_stopped else 0,
                        correct_count
                        ])
            if SAVE_HEATMAP:
                a.saveHeatmap(f"output/{SAVE_HEATMAP_FILE_NAME}/{util.fillZero(ep*(LIGHT_EXPOSURE_TIME + WAITING_TIME)+t+1,5)}.png")
            output_vector_list_prev.append(copy.deepcopy(a.h[-1]))

    f.close()
    print("\nDone")
    # clear_output()

convert output images into video